In [1]:
 suppressPackageStartupMessages({
    library(BiocGenerics)
    library(SummarizedExperiment)
    library(tximeta)
    library(rjson)
    library(tximport)
    library(dplyr)
    library(data.table)
    library(ggplot2)
    library(ggrepel)
    library("DESeq2")
    library("tidyr")
    library("tidyr")
    library("tibble")
    library("fgsea")
    library(apeglm)
    library(ggprism)
})


In [2]:
gene_map <- fread('../../../reference_files/gencode.v38.gene_transcript_type.tsv',header=FALSE)
colnames(gene_map) <- c("gene_id","transcript_id","gene_type","gene_symbol")
gene_map <- dplyr::select(data.frame(gene_map), gene_id, gene_type, gene_symbol) %>% distinct()

In [3]:
### we are interested in the effect of BHLHE40 OE in STIM conditions
files <- c("jurkat_control_1.gene_quants.sf",
           "jurkat_control_2.gene_quants.sf",
           "jurkat_control_3.gene_quants.sf",
           "jurkat_stim_1.gene_quants.sf",
           "jurkat_stim_2.gene_quants.sf",
           "jurkat_stim_3.gene_quants.sf")

txi.rna <- tximport(files, type = "salmon", txOut = TRUE)

txi.rna_se_counts <- SummarizedExperiment(assays=list(counts=txi.rna$counts,
                                                      lengths=txi.rna$length))

## Create a sampletable/metadata
condition = factor(c("ctrl", "ctrl", "ctrl",
                     "stim", "stim", "stim"))

meta <- data.frame(condition)
dds <- DESeqDataSetFromTximport(txi.rna, colData = meta, design = ~condition)
dds$condition <- relevel(dds$condition, "ctrl")
dds <- DESeq(dds)
#dds <- estimateSizeFactors(dds) genotypeOE.conditionstim
results <- results(dds, contrast = c("condition", "stim", "ctrl"))#name="")
resLFC <- lfcShrink(dds, coef="condition_stim_vs_ctrl", type="apeglm")
#results <- results(dds, name="genotypeOE.conditionstim")

reading in files with read_tsv

1 
Warning message:
"One or more parsing issues, see `problems()` for details"
2 
Warning message:
"One or more parsing issues, see `problems()` for details"
3 
Warning message:
"One or more parsing issues, see `problems()` for details"
4 
Warning message:
"One or more parsing issues, see `problems()` for details"
5 
Warning message:
"One or more parsing issues, see `problems()` for details"
6 
Warning message:
"One or more parsing issues, see `problems()` for details"


using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the n

In [4]:
res_table_tb <- resLFC %>%
  data.frame() %>%
  rownames_to_column(var="gene") %>% 
  as_tibble()

res_table_tb.dt <- data.table(res_table_tb)
res_table_tb.dt <- merge(res_table_tb.dt, gene_map, by.x = c("gene"), by.y = c("gene_id"))
res_table_tb.dt <- filter(res_table_tb.dt, gene_type == 'protein_coding')

res_table_tb.dt$gene <- sapply(strsplit(res_table_tb.dt$gene, "\\."),'[',1)
res_table_tb.dt_sig <- filter(res_table_tb.dt, (padj < 0.05))
res_table_tb.dt$gene_name <- ""
res_table_tb.dt_sig$gene_name <- ""


In [5]:

### list of ebox and gata factors
ebox_gata <- fread('ebox_gata.txt',header=FALSE)
colnames(ebox_gata) <- c("gene_name")

In [6]:
res_table_tb.dt_filt <- filter(res_table_tb.dt,
                               gene_symbol %in% ebox_gata$gene_name)

write.table(res_table_tb.dt_filt,
            'stim_v_wt_results.tsv',
            sep = '\t', col.names=TRUE,row.names=FALSE)

